In [8]:
import os
import sys
import pandas as pd
import requests
import time
import tqdm
import numpy as np
import json
from query import *

In [13]:
# Load .json file
with open('DGA_cURL.json', 'r') as file:
    jsondata = json.load(file)
cookies = jsondata['cookies']
headers = jsondata['headers']
data    = jsondata['data']

# Edit json file with the download id to a random key (for looping along ids/stations)
ds_data = pd.Series(data)
# ds_data = ds_data.rename({'filtroscirhform:j_idt177':'XXX'}) # <--- Change idt177 for the id of the station you downloaded.
ds_data

filtroscirhform                                                                  filtroscirhform
filtroscirhform:regionFieldSetId-value                                                      true
filtroscirhform:j_idt30-value                                            filtroscirhform:j_idt35
filtroscirhform:j_idt40                                                                       on
filtroscirhform:panelFiltroEstaciones-value                                                 true
filtroscirhform:region                                                                         5
filtroscirhform:selectBusqForEstacion                                                          1
filtroscirhform:cuenca                                                                        -1
filtroscirhform:estacion                                                                        
g-recaptcha-response                           03AFcWeA56zbaH31i8JB7btcbWSDvzr4ynhEcD-jbVvEIm...
filtroscirhform:j_idt100-value

In [14]:
# Download runoff from a specific station

url     = 'https://snia.mop.gob.cl/BNAConsultas/reportes'
years   = np.arange(1950, 2025,1)

variable = 'AlturayCaudalInstantaneo'
fpreffix = 'Qmax' 
errors   = []
corrupts = []


# Set station ID, Chile Political reigon and loop over years
id     = 416
region = 5
for k,yr in tqdm.tqdm(enumerate(years), total=len(years)):
    ofile = f'tmp/{fpreffix}_{region}Region_idt{id}_{yr}.xls'
    if os.path.isfile(ofile):
        pass
    else:
        
        ndata = ds_data.copy()
        ndata = ndata.rename({'XXX':f'filtroscirhform:j_idt{id}'})
        ndata['filtroscirhform:region'] = region
        ndata['filtroscirhform:fechaDesdeInputDate'] = f'01/01/{yr}'
        ndata['filtroscirhform:fechaHastaInputDate'] = f'31/12/{yr}'
        ndata = dict(ndata)
        try:
            value = DGA_makerequest(ofile, url, cookies, headers, ndata)
            if value == False:
                corrupts.append((region, id, yr))
        except Exception as e:
            print('ERROR:', e)
            errors.append((region, id, yr))

100%|██████████| 75/75 [13:22<00:00, 10.69s/it]


In [13]:
# Download anual-max precipitation for multiple stations

url = 'https://snia.mop.gob.cl/BNAConsultas/reportes'
ids     = np.arange(181,630,5)
ids     = np.hstack([177, ids])


variable = 'PrecipitacionesMaximasEn24Horas'
fpreffix = 'PrecipMax' 
errors   = []
corrupts = []

# Set region and loop
region = 12

# Loop over stations
for j,id in tqdm.tqdm(enumerate(ids), total=len(ids)):
    #Loop over years
    for k,yr in enumerate([1910,1950,1990]):
        ofile = f'tmp/{fpreffix}_{region}Region_idt{id}_{yr}.xls'
        if os.path.isfile(ofile):
            pass
        else:
            
            ndata = ds_data.copy()
            ndata = ndata.rename({'XXX':f'filtroscirhform:j_idt{id}'})
            ndata['filtroscirhform:region'] = region
            ndata['filtroscirhform:fechaDesdeInputDate'] = f'01/01/{yr}'
            ndata['filtroscirhform:fechaHastaInputDate'] = f'31/12/{yr+39}'
            ndata = dict(ndata)
            try:
                value = DGA_makerequest(ofile, url, cookies, headers, ndata)
                if value == False:
                    corrupts.append((region, id, yr))
            except Exception as e:
                print('ERROR:', e)
                errors.append((region, id, yr))

  2%|▏         | 2/91 [00:11<08:32,  5.75s/it]

Request ended with: 502 Trying again...


 21%|██        | 19/91 [01:59<08:05,  6.75s/it]

Request ended with: 502 Trying again...


 40%|███▉      | 36/91 [06:06<05:36,  6.12s/it]

...

 87%|████████▋ | 79/91 [10:23<01:08,  5.72s/it]

Request ended with: 502 Trying again...


 89%|████████▉ | 81/91 [10:39<01:08,  6.81s/it]

Request ended with: 502 Trying again...


100%|██████████| 91/91 [11:35<00:00,  7.65s/it]


In [27]:
# Download daily precipitation for multiple stations
url = 'https://snia.mop.gob.cl/BNAConsultas/reportes'
ids     = np.arange(181,630,5)
ids     = np.hstack([177, ids])
# ids = [186, 211, 296, 366, 466, 556, 196, 216, 351, 386, 491, 566]
years   = np.arange(1910, 2025,3)


variable = 'PrecipitacionesDiarias'
fpreffix = 'Precip24h' 
errors   = []
corrupts = []
region = 7
for j,id in tqdm.tqdm(enumerate(ids), total=len(ids)):
    for k,yr in enumerate(years):
        ofile = f'tmp/{fpreffix}_{region}Region_idt{id}_{yr}.xls'
        if os.path.isfile(ofile):
            pass
        else:
            ndata = ds_data.copy()
            ndata = ndata.rename({'XXX':f'filtroscirhform:j_idt{id}'})
            ndata['filtroscirhform:region'] = region
            ndata['filtroscirhform:fechaDesdeInputDate'] = f'01/01/{yr}'
            ndata['filtroscirhform:fechaHastaInputDate'] = f'31/12/{yr+3}'
            ndata = dict(ndata)
            try:
                value = DGA_makerequest(ofile, url, cookies, headers, ndata)
                if value == False:
                    corrupts.append((region, id, yr))
            except Exception as e:
                print('ERROR:', e)
                errors.append((region, id, yr))

  0%|          | 0/91 [00:00<?, ?it/s]

Request ended with: 502 Trying again...


  9%|▉         | 8/91 [05:16<53:26, 38.63s/it]  

Request ended with: 502 Trying again...


 10%|▉         | 9/91 [06:10<59:15, 43.36s/it]

...

 95%|█████████▍| 86/91 [1:47:06<06:14, 74.98s/it]

Request ended with: 502 Trying again...


 97%|█████████▋| 88/91 [1:49:33<03:42, 74.05s/it]

Request ended with: 502 Trying again...
Request ended with: 502 Trying again...


100%|██████████| 91/91 [1:53:14<00:00, 74.66s/it]
